In [ ]:
import json
import os
import tempfile
import pandas as pd
import google.generativeai as genai
import gradio as gr
from google.colab import userdata

In [ ]:
def create_client(api_key: str):
    """Initialize Gemini with the current stable Flash model"""
    genai.configure(api_key=api_key)

    # gemini-2.5-flash is the stable production model as of late 2025/2026.
    # gemini-flash-latest is an alias that always points to the newest stable Flash model.
    model = genai.GenerativeModel(
        model_name="gemini-2.5-flash",
        generation_config={"response_mime_type": "application/json"}
    )
    return model

In [ ]:
# 2. Single Batch Generation
def generate_synthetic_data(
    api_key: str,
    schema_description: str,
    num_records: int,
    example_format: str = ""
) -> tuple:
    """Generate synthetic records using Gemini JSON mode"""
    try:
        model = create_client(api_key)

        example_section = f"\n\nExample format:\n{example_format}" if example_format else ""

        # System-style prompt
        prompt = f"""You are a data generation expert.
Generate {num_records} synthetic data records based on this schema:
{schema_description}
{example_section}

Requirements:
- Return a JSON object with a key "records" containing an array of objects.
- Each object must match the schema provided.
- Ensure data is realistic, diverse, and correctly typed.
- Output ONLY the JSON object.

Generate exactly {num_records} records."""

        # Call Gemini
        response = model.generate_content(prompt)

        # Parse result
        raw_data = json.loads(response.text)

        # Extract the list from the "records" key
        if isinstance(raw_data, dict) and "records" in raw_data:
            data = raw_data["records"]
        else:
            # Fallback: find the first list in the dictionary
            data = next((v for v in raw_data.values() if isinstance(v, list)), [])

        df = pd.DataFrame(data).head(num_records)

        # Save to temp CSV
        fd, temp_path = tempfile.mkstemp(suffix='.csv', prefix='gemini_synth_')
        os.close(fd)
        df.to_csv(temp_path, index=False)

        return df, f"✅ Successfully generated {len(df)} records!", temp_path

    except Exception as e:
        return None, f"❌ Gemini Error: {str(e)}", None

In [ ]:
# 3. Batch Logic (for larger datasets)
def generate_batch_data(
    api_key: str,
    schema_description: str,
    total_records: int,
    example_format: str = "",
    batch_size: int = 50
) -> tuple:
    """Consolidate multiple Gemini API calls into one DataFrame"""
    all_data = []
    batches = (total_records + batch_size - 1) // batch_size

    print(f"Starting generation of {total_records} records in {batches} batches...")

    for i in range(batches):
        remaining = total_records - len(all_data)
        records_in_batch = min(batch_size, remaining)

        df_batch, status, csv_path = generate_synthetic_data(
            api_key, schema_description, records_in_batch, example_format
        )

        if df_batch is not None:
            all_data.extend(df_batch.to_dict('records'))
            if os.path.exists(csv_path): os.remove(csv_path) # Cleanup temp batch files
            print(f"Batch {i+1}/{batches} complete...")
        else:
            return None, f"❌ Failed at batch {i+1}: {status}", None

    # Final Consolidation
    final_df = pd.DataFrame(all_data)
    fd, temp_path = tempfile.mkstemp(suffix='.csv', prefix='final_gemini_data_')
    os.close(fd)
    final_df.to_csv(temp_path, index=False)

    return final_df, f"✅ Generated {len(final_df)} records in {batches} batches!", temp_path

In [ ]:
google_api = userdata.get('GOOGLE_API_KEY')
# Create Gradio Interface
def create_interface():
    with gr.Blocks(title="Synthetic Dataset Generator", theme=gr.themes.Base()) as demo:
        gr.Markdown("""
        # 🤖 Synthetic Dataset Generator
        ### Powered by Google Gemini 1.5 Flash

        Create custom synthetic datasets by describing your schema. Gemini will generate realistic data matching your specifications using high-speed JSON mode.
        """)

        with gr.Row():
            with gr.Column(scale=1):
                # Show API key input only if not found in environment/secrets
                if not google_api:
                    api_key_input = gr.Textbox(
                        label="Gemini API Key",
                        type="password",
                        placeholder="AIza...",
                        info="API key not found in Colab Secrets"
                    )
                else:
                    api_key_input = gr.Textbox(
                        label="Gemini API Key",
                        type="password",
                        value=google_api,
                        placeholder="Loaded from Secrets",
                        info="✅ API key loaded from environment",
                        interactive=False
                    )

                schema_input = gr.Textbox(
                    label="Data Schema Description",
                    placeholder="""Example: Generate customer data with:
- name (full name)
- email (valid email address)
- age (between 18-80)
- city (US cities)
- purchase_amount (between $10-$1000)
- join_date (dates in 2023-2024)""",
                    lines=10
                )

                example_input = gr.Textbox(
                    label="Example Format (Optional JSON)",
                    placeholder="""{"name": "John Doe", "email": "john@example.com", "age": 35, "city": "New York"}""",
                    lines=4
                )

                num_records = gr.Slider(
                    minimum=1,
                    maximum=500,
                    value=20,
                    step=1,
                    label="Number of Records"
                )

                generate_btn = gr.Button("🚀 Generate Dataset", variant="primary")

            with gr.Column(scale=2):
                status_output = gr.Textbox(label="Status", lines=2)
                dataframe_output = gr.Dataframe(
                    label="Generated Dataset Preview",
                    wrap=True
                )
                csv_output = gr.File(label="Download Full CSV", type="filepath")

        # Examples
        gr.Markdown("### 📝 Example Schemas")
        gr.Examples(
            examples=[
                [
                    """Generate employee records with:
- employee_id (format: EMP001, EMP002, etc.)
- name (full name)
- department (Engineering, Sales, Marketing, HR, Finance)
- salary (between $40,000-$150,000)
- hire_date (between 2020-2024)
- performance_rating (1-5)""",
                    20
                ],
                [
                    """Generate e-commerce product data with:
- product_id (format: PRD-XXXX)
- product_name (creative product names)
- category (Electronics, Clothing, Home, Books, Sports)
- price (between $5-$500)
- stock_quantity (between 0-1000)
- rating (1.0-5.0)""",
                    25
                ],
                [
                    """Generate student records with:
- student_id (format: STU2024XXX)
- name (full name)
- major (Computer Science, Biology, Business, Arts, Engineering)
- gpa (2.0-4.0)
- year (Freshman, Sophomore, Junior, Senior)""",
                    30
                ]
            ],
            inputs=[schema_input, num_records]
        )

        def generate_wrapper(api_key, schema, num_rec, example):
            # Prioritize input field, then fallback to environment/secrets
            final_api_key = api_key or google_api

            if not final_api_key:
                return None, "❌ Please provide your Gemini API key (via Colab Secrets or the input field)", None
            if not schema:
                return None, "❌ Please describe your data schema", None

            # For larger datasets, use batch generation (defined in previous steps)
            if num_rec > 50:
                return generate_batch_data(final_api_key, schema, num_rec, example)
            else:
                return generate_synthetic_data(final_api_key, schema, num_rec, example)

        generate_btn.click(
            fn=generate_wrapper,
            inputs=[api_key_input, schema_input, num_records, example_input],
            outputs=[dataframe_output, status_output, csv_output]
        )

        gr.Markdown("""
        ---
        ### 💡 Tips:
        - **JSON Mode**: Gemini 1.5 Flash is highly optimized for structured data.
        - **Specifics**: Be specific about ranges (e.g., "Age between 21 and 65").
        - **Batching**: Large requests (>50 records) are automatically split to ensure quality.
        - **API Cost**: Gemini 1.5 Flash is currently free (within rate limits) in many regions.

        ### 🔑 API Key Setup in Colab:
        1. Click the **Key icon (Secrets)** on the left sidebar.
        2. Add a new secret named `GEMINI_API_KEY`.
        3. Paste your key from [Google AI Studio](https://aistudio.google.com/).
        4. Enable "Notebook access" for that key.
        """)

    return demo

In [ ]:
from gradio import themes
# 5. Launch the App
if __name__ == "__main__":
    demo = create_interface()
    demo.launch(share=True, debug=True)

/tmp/ipython-input-3728467288.py:4: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(title="Synthetic Dataset Generator", theme=gr.themes.Base()) as demo:


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://b4da2027e1ee849ba0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
